In [1]:
from brian2 import *
import numpy as np
import pickle

In [2]:
s = np.genfromtxt('stimulus.csv', delimiter=',')


In [3]:
# network param.
M = 25*5
L = 4
N = M*L;
N_E = N
N_readout = 400

n_trials = 30

# neuron param.
tau_mem = 20*ms
thresh = 20 *mV
V_reset = 10*mV
V_rest = 0*mV
#thresh = -55 *mV
#V_reset = -65*mV
#V_rest = -75*mV
tau_rp = 2*ms
D = 1.5 *ms # Delay (unused at the moment)

eqs = '''
dv/dt = -(v-V_rest)/tau_mem : volt (unless refractory)
'''

# synapse param.
J_E = 0.2*mV
J_EE = 10*J_E
J_I = J_EE
#nu_thr = thresh / (J_E * tau_mem)

theta_0 = np.linspace(0,1,N + 1)
theta_0 = theta_0[:-1] #remove last element (circular stimulus variable)

# preferred locations of readout layer neurons
readout_theta_0 = np.linspace(0,1,N_readout + 1)
readout_theta_0 = readout_theta_0[0:-1] #same as for theta0

max_input = 5000* Hz #Heuristically set

#controlls width of input modulation to input layer
w = 0.3 
# Spatial frequency (1/(R*spatial period)) of input layer neurons
f_1 = 1
f_2 = 2
f_3 = 3
f_4 = 4

theta_0_module_1 = np.linspace(0,1/f_1, M + 1) #only sample the preferred conditions within one spatial period
theta_0_module_1 = theta_0_module_1[:-1] #remove last element (circular stimulus variable -> it is same as first)

theta_0_module_2 = np.linspace(0,1/f_2, M + 1) #only sample the preferred conditions within one spatial period
theta_0_module_2 = theta_0_module_2[:-1] #remove last element (circular stimulus variable -> it is same as first)

theta_0_module_3 = np.linspace(0,1/f_3, M + 1) #only sample the preferred conditions within one spatial period
theta_0_module_3 = theta_0_module_3[:-1] #remove last element (circular stimulus variable -> it is same as first)

theta_0_module_4 = np.linspace(0,1/f_4, M + 1) #only sample the preferred conditions within one spatial period
theta_0_module_4 = theta_0_module_4[:-1] #remove last element (circular stimulus variable -> it is same as first)

# Poisson input rates to the input layer
# Baseline input is 0.85*5000 = 4250 just enough to make neurons spike a few times, rest 0.15*5000 = 750 is stimulus modulation
input_rates_module_1 = lambda theta: (0.15*exp(1/w*(cos(2*pi*f_1*(theta-theta_0_module_1)) - 1)) + 0.85 )*max_input
input_rates_module_2 = lambda theta: (0.15*exp(1/w*(cos(2*pi*f_2*(theta-theta_0_module_2)) - 1)) + 0.85 )*max_input
input_rates_module_3 = lambda theta: (0.15*exp(1/w*(cos(2*pi*f_3*(theta-theta_0_module_3)) - 1)) + 0.85 )*max_input
input_rates_module_4 = lambda theta: (0.15*exp(1/w*(cos(2*pi*f_4*(theta-theta_0_module_4)) - 1)) + 0.85 )*max_input

In [4]:
#activity input to input layer
TC_vec_1 = [list(input_rates_module_1(min(1,max(s[i],0)))) for i in range(len(s[:]))]
TC_vec_2 = [list(input_rates_module_2(min(1,max(s[i],0)))) for i in range(len(s[:]))]
TC_vec_3 = [list(input_rates_module_3(min(1,max(s[i],0)))) for i in range(len(s[:]))]
TC_vec_4 = [list(input_rates_module_4(min(1,max(s[i],0)))) for i in range(len(s[:]))]

#these are the equivalents to "stimulus" variable in the Gaussian network case
input_module_1 = TimedArray(TC_vec_1, dt=0.1*ms)
input_module_2 = TimedArray(TC_vec_2, dt=0.1*ms)
input_module_3 = TimedArray(TC_vec_3, dt=0.1*ms)
input_module_4 = TimedArray(TC_vec_4, dt=0.1*ms)

In [5]:
for k in range(0,n_trials):
    start_scope()

    defaultclock.dt = 0.1 * ms
    P_1 = PoissonGroup(M, rates='input_module_1(t, i)')
    P_2 = PoissonGroup(M, rates='input_module_2(t, i)')
    P_3 = PoissonGroup(M, rates='input_module_3(t, i)')
    P_4 = PoissonGroup(M, rates='input_module_4(t, i)')

    MP_1 = SpikeMonitor(P_1)
    MP_2 = SpikeMonitor(P_2)
    MP_3 = SpikeMonitor(P_3)
    MP_4 = SpikeMonitor(P_4)

    run(1000*ms)
    # And keep a copy of those spikes
    spikes_i_M_1 = MP_1.i
    spikes_t_M_1 = MP_1.t

    spikes_i_M_2 = MP_2.i
    spikes_t_M_2 = MP_2.t

    spikes_i_M_3 = MP_3.i
    spikes_t_M_3 = MP_3.t

    spikes_i_M_4 = MP_4.i
    spikes_t_M_4 = MP_4.t
    
    start_scope()
    defaultclock.dt = 0.1 * ms
    P = PoissonGroup(N_readout, rates=0.85*max_input)
    MP = SpikeMonitor(P)
    run(1000*ms)
    # And keep a copy of those spikes
    spikes_i_readout_base = MP.i
    spikes_t_readout_base = MP.t

    start_scope()
    defaultclock.dt = 0.1 * ms
    module_1 = NeuronGroup(M, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )

    module_2 = NeuronGroup(M, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )

    module_3 = NeuronGroup(M, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )

    module_4 = NeuronGroup(M, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )

    readout_neurons = NeuronGroup(N_readout, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )

    module_1.v = 10*(1 + rand(M))*mV # initial value
    module_2.v = 10*(1 + rand(M))*mV # initial value
    module_3.v = 10*(1 + rand(M))*mV # initial value
    module_4.v = 10*(1 + rand(M))*mV # initial value

    readout_neurons.v = 10*(1 + rand(N_readout))*mV # initial value


    #Synapses input activity -> input layer
    SGG_1 = SpikeGeneratorGroup(M, spikes_i_M_1, spikes_t_M_1)
    external_syn_M_1 = Synapses(SGG_1, target=module_1, on_pre='v += J_E')
    external_syn_M_1.connect(j='i') #connect 1-to-1

    SGG_2 = SpikeGeneratorGroup(M, spikes_i_M_2, spikes_t_M_2)
    external_syn_M_2 = Synapses(SGG_2, target=module_2, on_pre='v += J_E')
    external_syn_M_2.connect(j='i') #connect 1-to-1

    SGG_3 = SpikeGeneratorGroup(M, spikes_i_M_3, spikes_t_M_3)
    external_syn_M_3 = Synapses(SGG_3, target=module_3, on_pre='v += J_E')
    external_syn_M_3.connect(j='i') #connect 1-to-1

    SGG_4 = SpikeGeneratorGroup(M, spikes_i_M_4, spikes_t_M_4)
    external_syn_M_4 = Synapses(SGG_4, target=module_4, on_pre='v += J_E')
    external_syn_M_4.connect(j='i') #connect 1-to-1

    #Synapses input layer -> readout layer (distance dependent EPSPs - in stimulus space)
    read_out_syn_M_1 = Synapses(module_1, readout_neurons, 'epsp : volt', on_pre='v += epsp', delay = D)
    read_out_syn_M_2 = Synapses(module_2, readout_neurons, 'epsp : volt', on_pre='v += epsp', delay = D)
    read_out_syn_M_3 = Synapses(module_3, readout_neurons, 'epsp : volt', on_pre='v += epsp', delay = D)
    read_out_syn_M_4 = Synapses(module_4, readout_neurons, 'epsp : volt', on_pre='v += epsp', delay = D)

    read_out_syn_M_1.connect()
    read_out_syn_M_2.connect()
    read_out_syn_M_3.connect()
    read_out_syn_M_4.connect()

    FWHM = 1/N_readout;
    sigma = (FWHM/2) / sqrt(2*log(2));
    # Set the EPSPs
    w_readout = (pi*FWHM)**2/(2*log(2))
    read_out_syn_M_1.epsp = 'exp(1/(w_readout)*(cos(2*pi*f_1*(1/M*i/f_1 - 1/N_readout*j))-1))*J_EE'
    read_out_syn_M_2.epsp = 'exp(1/(w_readout)*(cos(2*pi*f_2*(1/M*i/f_2 - 1/N_readout*j))-1))*J_EE'
    read_out_syn_M_3.epsp = 'exp(1/(w_readout)*(cos(2*pi*f_3*(1/M*i/f_3 - 1/N_readout*j))-1))*J_EE'
    read_out_syn_M_4.epsp = 'exp(1/(w_readout)*(cos(2*pi*f_4*(1/M*i/f_4 - 1/N_readout*j))-1))*J_EE'

    #Synapses baseline input to readout layer
    SGG_readout = SpikeGeneratorGroup(N_readout, spikes_i_readout_base, spikes_t_readout_base)
    external_syn_readout = Synapses(SGG_readout, target=readout_neurons, on_pre='v += J_E')
    external_syn_readout.connect(j='i') #connect 1-to-1

    #Winner take all style inhibitory synapses between readout layer (distance dependent, "nearer" -> less inhibition)
    read_out_syn_inhib = Synapses(readout_neurons, readout_neurons, 'ipsp : volt', on_pre='v += ipsp', delay = D)
    read_out_syn_inhib.connect()
    read_out_syn_inhib.ipsp = '-abs(sin(pi*(1/N_readout*i-1/N_readout*j)))*J_I' # '-(1 - exp(1/0.15*(cos(2*pi*(1/N_readout*i-1/N_readout*j)) - 1)))*J_I' 

    spike_monitor_M_1 = SpikeMonitor(module_1)
    spike_monitor_M_2 = SpikeMonitor(module_2)
    spike_monitor_M_3 = SpikeMonitor(module_3)
    spike_monitor_M_4 = SpikeMonitor(module_4)

    spike_monitor_readout = SpikeMonitor(readout_neurons)

    run(1000*ms)
    t_idx = (1/defaultclock.dt)*spike_monitor_readout.t[:]
    t_idx = t_idx.astype(int)
    
    data_1 = spike_monitor_M_1.get_states(['t','i','count'])
    data_2 = spike_monitor_M_2.get_states(['t','i','count'])
    data_3 = spike_monitor_M_3.get_states(['t','i','count'])
    data_4 = spike_monitor_M_4.get_states(['t','i','count'])
    
    data_readout = spike_monitor_readout.get_states(['t','i','count'])
    with open('./data/Periodic_population/data_TC_layer_{}.pickle'.format(k), 'wb') as f:
        pickle.dump(data_1, f)
        pickle.dump(data_2, f)
        pickle.dump(data_3, f)
        pickle.dump(data_4, f)
    with open('./data/Periodic_population/data_readout_layer_{}.pickle'.format(k), 'wb') as f:
        pickle.dump(data_readout, f) 

WARNING    Cannot use Cython, a test compilation failed: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/ (DistutilsPlatformError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]
